In [ ]:
!pip install yfinance --upgrade --no-cache-dir

import math
from datetime import datetime
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as Dash
import yfinance as yf
yf.pdr_override()
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pegar os dados do Bitcoin com o Yfinance
Base_Dados = web.get_data_yahoo('BTC-USD', start='2021-01-01')
Base_Dados.describe()

In [ ]:
#px.line(Base_Dados, y='Close', title= 'Preço de Fechamento').show()

In [ ]:
# MM do fechamento
Media_Movel = Base_Dados['Close'].rolling(9).mean()
Media_Movel_Tendencia = Base_Dados['Close'].rolling(21).mean()

Figura = Dash.Figure()

Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Base_Dados.Close, 
                    mode='lines',
                    name='Fechamento',
                    marker_color = '#FF7F0E'))

Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel,
                    mode='lines',
                    name='MMS 9', opacity=0.5,
                    marker_color = '#2CA02C',
                    ))

Figura.add_trace(Dash.Scatter(x = Base_Dados.index, y = Media_Movel_Tendencia,
                    mode='lines',
                    name='MMS 21', opacity=0.5,
                    marker_color = '#D62728'))

Figura.update_layout(
    title='Histórico de Preço',
    titlefont_size = 28,
    
    # eixo X
    xaxis = dict( 
        title='Período Histórico',
        titlefont_size=16, 
        tickfont_size=14),

    height = 500, 
    
    # eixo y
    yaxis=dict(
        title='Preço Bitcoint', 
        titlefont_size=16, 
        tickfont_size=14),  
    
    # legenda
    legend=dict(
        y=1, x=1,
        bgcolor='rgba(255, 255, 255, 0)',
        bordercolor='rgba(255, 255, 255, 0)'))

Figura.show()

In [ ]:
# escolanemnto
Funcao_MinMAx = MinMaxScaler(feature_range=(0,1) )

Dados_Treino = Base_Dados.filter(['Close'])

Dados_Treino_Escalados = Funcao_MinMAx.fit_transform(Dados_Treino)

In [131]:
x_treinamento = []
y_treinamento = []

for Loop in range(60, len(Dados_Treino_Escalados)):

    Filtrando_Amostra_Treinamento_x = Dados_Treino_Escalados[ Loop-60 : Loop, 0 ]
    x_treinamento.append( Filtrando_Amostra_Treinamento_x )

    Filtrando_Amostra_Treinamento_y = Dados_Treino_Escalados[Loop, 0] 
    y_treinamento.append( Filtrando_Amostra_Treinamento_y )
    
x_treinamento, y_treinamento = np.array(x_treinamento), np.array(y_treinamento) 

x_treinamento = np.reshape(x_treinamento, (x_treinamento.shape[0], x_treinamento.shape[1], 1))

In [ ]:
Modelo = Sequential()

# LSTM - Long Short-Term Memory
Modelo.add(LSTM(50, return_sequences = True, 
                input_shape = (x_treinamento.shape[1], 1)))
Modelo.add(LSTM(50, return_sequences = False))

# Adicionando as camadas na rede neural
Modelo.add(Dense(25))
Modelo.add(Dense(1))
Modelo.compile(optimizer = 'adam', loss = 'mean_squared_error')

Modelo.fit(x_treinamento, y_treinamento, batch_size = 1, epochs = 15)

In [ ]:
# Definindo amostra para ser testada

Dados_Fechamento_Valores_Tamanho = math.ceil( len(Dados_Treino) * .8)
Dados_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho - 60: , :]

x_teste = []

y_Teste = Dados_Treino_Escalados[Dados_Fechamento_Valores_Tamanho:, :]

for Loop in range (60, len(Dados_Teste)):
    x_teste.append(Dados_Teste[Loop - 60:Loop, 0])

x_teste = np.array(x_teste)

x_teste = np.reshape(x_teste, (x_teste.shape[0], x_teste.shape[1], 1))


Previsoes = Modelo.predict(x_teste)

# RSME
rsme = np.sqrt(np.mean(Previsoes - y_Teste) ** 2)
print('Erro Quadrático Médio:', f"{rsme:.2%}")

# Revertendo para escalas reais 
Previsoes = Funcao_MinMAx.inverse_transform(Previsoes)

In [177]:
# real x modelo
Validação = Dados_Treino[Dados_Fechamento_Valores_Tamanho:]

Validação['Previsões'] = Previsoes

In [ ]:
# Gráfico Dinâmico

Figura = Dash.Figure()

Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Close, 
                    mode='lines',
                    name='Fechamento',
                    marker_color = '#FF7F0E',
                    ))

Figura.add_trace(Dash.Scatter(x = Validação.index, y = Validação.Previsões,
                    mode='lines',
                    name='Previsão',
                    marker_color = '#2CA02C',
                    ))

Figura.update_layout(
    title='Realizado vs Modelo', 
    titlefont_size = 28,
    
    # eixo X
    xaxis = dict( 
        title='Período Histórico', 
        titlefont_size=16, 
        tickfont_size=14),  

    height = 500, 
    
    # eixo y
    yaxis=dict(
        title='Preço do Bitcoin', 
        titlefont_size=16, 
        tickfont_size=14), 
    
    # legenda
    legend=dict(
        y=1, x=1, 
        bgcolor='rgba(255, 255, 255, 0)', 
        bordercolor='rgba(255, 255, 255, 0)'))

Figura.show()

In [ ]:
fech, Prev = Validação.loc[datetime.today().strftime('%Y-%m-%d')]
percentual = (Prev / Fech - 1)

if percentual > 0:
   print("Recomendação de COMPRA, valor previsto", f"{percentual:.2%}", "acima do fechamento!")
else:
   print("Recomendação de VENDA, valor previsto", f"{percentual:.2%}", "abaixo do fechamento!")